<a href="https://colab.research.google.com/github/tonyyoon123/MachineLearning/blob/master/Assignment10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Import Libaries*

In [0]:
import matplotlib.pyplot as plt
import numpy as np

*Input Data*

In [0]:
file_data   = "mnist.csv"
handle_file = open(file_data, "r")
data        = handle_file.readlines()
handle_file.close()

size_row    = 28    # height of the image
size_col    = 28    # width of the image

num_image   = len(data)
count       = 0     # count for the number of images

# normalize the values of the input data to be [0, 1]
def normalize(data):

    data_normalized = (data - min(data)) / (max(data) - min(data))

    return(data_normalized)

# example of distance function between two vectors x and y
def distance(x, y):

    d = (x - y) ** 2
    s = np.sum(d)
    # r = np.sqrt(s)

    return(s)

def sigmoid(z):
  return 1 / ( 1 + np.exp(-z))

# make a matrix each column of which represents an images in a vector form
list_image  = np.empty((size_row * size_col, num_image), dtype=float)
list_label  = np.empty(num_image, dtype=int)

for line in data:
  line_data   = line.split(',')
  label       = line_data[0]
  im_vector   = np.asfarray(line_data[1:])    
  list_label[count]       = label
  list_image[:, count]    = im_vector
  count += 1

In [0]:
#----------------------------------------#
#여기서 부터는 내 코드#
#---------------------------------------#
# Train(1000), Test(9000)
train_imgs = np.empty((1000, size_row * size_col), dtype=float)
test_imgs = np.empty((9000, size_row * size_col), dtype=float)

train_labels = np.empty(1000, dtype=int)
test_labels = np.empty(9000, dtype=int)

for i in range(num_image):
  if i < 1000:
    train_labels[i] = list_label[i]
    train_imgs[i] = list_image[:, i]
  else:
    test_labels[i-9000] = list_label[i]
    test_imgs[i-9000] = list_image[:, i]


# One hot encoding our labels
train_targets = np.array(train_labels).astype(np.int)
train_labels_one_hot = np.eye(np.max(train_targets) + 1)[train_targets]

test_targets = np.array(test_labels).astype(np.int)
test_labels_one_hot = np.eye(np.max(test_targets) + 1)[test_targets]

train_labels_one_hot[train_labels_one_hot==0] = 0.01
train_labels_one_hot[train_labels_one_hot==1] = 0.99

test_labels_one_hot[test_labels_one_hot==0] = 0.01
test_labels_one_hot[test_labels_one_hot==1] = 0.99

class Network:
  def __init__(self):
    self.learning_rate = 0.1
    self.decay = 0.6
    self.w1 = np.random.randn(784, 196)
    self.b1 = np.zeros(self.w1.shape[1])  
    self.w2 = np.random.randn(self.w1.shape[1], 49)
    self.b2 = np.zeros(self.w2.shape[1])  
    self.w3 = np.random.randn(self.w2.shape[1], 10)
    self.b3 = np.zeros(self.w3.shape[1])
    self.n= ((self.w1.shape[0] * self.w1.shape[1]) + 
              (self.w2.shape[0] * self.w2.shape[1]) + 
              (self.w3.shape[0] * self.w3.shape[1]))

  def forward_prop(self):
    x = np.dot(self.x, self.w1) + self.b1
    self.X = sigmoid(x)
    y = np.dot(self.X, self.w2) + self.b2
    self.Y = sigmoid(y)
    z = np.dot(self.Y, self.w3) + self.b3
    self.Z = sigmoid(Z)      

  #def cost_function(self, y_pred, y):
  #  m = y.shape[1]    
  #  cost = np.mean(np.sum(-y*np.log(y_pred), axis=1))
  #  return cost

  def back_prop(self):        
    errors = self.y - self.Z            
    self.w3 += (self.learning_rate * np.dot(self.Y.T, errors) + ((self.decay / self.n) * self.w3))
    self.b3 += self.learning_rate * np.sum(errors, axis=0)        

    errors = np.dot(errors, self.w3.T)   
    errors = np.multiply(errors, np.multiply(self.Y, (1.0 - self.Y)))        
    self.w2 += (self.learning_rate * np.dot(self.X.T, errors) +((self.decay / self.n) * self.w2))
    self.b2 += self.learning_rate * np.sum(errors, axis=0)        

    errors = np.dot(errors, self.w2.T)
    errors = np.multiply(errors, np.multiply(self.X, (1.0 - self.X)))
    self.w1 += (self.learning_rate * np.dot(self.x.T, errors)+ ((self.decay / self.n) * self.w1))
    self.b1 += self.learning_rate * np.sum(errors, axis=0)    
    
  def train(self, x, y):
    x = np.array(x, ndmin=2)
    y = np.array(y, ndmin=2)
    self.forward_prop()
    self.back_prop()

  def predict(self, x):
    self.forward_prop()
    return self.Z

  def evaluate(self, x, y):
    corrects, wrongs = 0, 0
    for i in range(len(x)):
      res = self.predict(x[i])
      res_max = res.argmax()
      if res_max == y[i]:
        corrects += 1        
      else:
        wrongs += 1        
    return corrects, wrongs



model = Network()
epochs = 10
costs_train = []
costs_test = []
accuracys_train = []
accuracys_test = []

for epoch in range(epochs):
  print("epoch: ", epoch+1)
  for i in range(len(train_imgs)):
    model.train(train_imgs[i], train_labels_one_hot[i])  
  
corrects, wrongs = model.evaluate(train_imgs, train_labels)  
accuracys_train.append(corrects / ( corrects + wrongs))
costs_train.append(model.cost_function(model.predict(train_imgs), train_labels_one_hot))  
corrects, wrongs = model.evaluate(test_imgs, test_labels)  
accuracys_test.append(corrects / ( corrects + wrongs))
costs_test.append(model.cost_function(model.predict(test_imgs), test_labels_one_hot))


plt.plot(costs_train, c='b', label='cost train-set')
plt.plot(costs_test, c='r', label='cost test-set')
plt.xlabel("Interation")
plt.ylabel("Cost")
plt.legend()
plt.show()

#--------------------------------#
plt.plot(accuracys_train, c='b', label='accuracy train-set')
plt.plot(accuracys_test, c='r', label='accuracy test-set')
plt.xlabel("Interation")
plt.ylabel("Accuracy")
plt.show()
#-------------------------------#
print('Final training accuracy: ',accuracys_train[9]*100, '%')
print('Final testing accuracy: ',accuracys_test[9]*100, '%')